In [40]:
from neo4j import GraphDatabase
import json
from py2neo import Graph, Node, Relationship, NodeMatcher
import spacy

In [13]:
graph = Graph("neo4j+s://a6801d44.databases.neo4j.io", user="neo4j", password="049h8PxTvW314YzhT1mxjNfkLpTLewtTG7FEZMj0vwY")

In [16]:
def create_node_and_relationship(sentence, subjects, objects, relationships):
    # Create nodes for subjects and objects
    for subject in subjects:
        graph.create(Node("Subject", name=subject[0]))
    for obj in objects:
        graph.create(Node("Object", name=obj[0]))

    # Create relationships between subjects and objects
    min_length = min(len(subjects), len(objects), len(relationships))
    for i in range(min_length):
        graph.create(Relationship(Node("Subject", name=subjects[i][0]), relationships[i], Node("Object", name=objects[i][0])))

In [17]:
with open("processed_entity_data.txt", "r") as file:
    json_data = json.load(file)

for data_entry in json_data:
    create_node_and_relationship(data_entry["sentence"], data_entry["subjects"], data_entry["objects"], data_entry["relationships"])

In [32]:
def answer_question(question):
    # Connect to your Neo4j database
    graph = Graph("neo4j+s://a6801d44.databases.neo4j.io", user="neo4j", password="049h8PxTvW314YzhT1mxjNfkLpTLewtTG7FEZMj0vwY")

 # Tokenize and process the question using spaCy
    question_doc = nlp(question)

    # Extract relevant information from the question
    question_subjects = [token.text.lower() for token in question_doc if token.dep_ in ("nsubj", "nsubjpass")]

    # Query the graph for nodes matching the subject
    for question_subject in question_subjects:
        print(f"Processing subject: {question_subject}")
        matching_nodes = graph.nodes.match("Subject", name=question_subject).limit(1)

        # Print the relevant information
        if matching_nodes:
            for node in matching_nodes:
                print(f"Matching node found: {node}")
                out_relations = graph.match((node, None), r_type="*", limit=1)
                if out_relations:
                    for rel in out_relations:
                        print(f"Outgoing relationship: {rel.type}, Target node: {rel.end_node}")
                else:
                    print(f"No outgoing relationships for node {node}")
        else:
            print(f"No matching nodes found in the graph for subject: {question_subject}")



In [36]:
nlp = spacy.load("en_core_web_sm")

In [33]:
question = "Who won a gold medal in the Bangabandhu Sheikh Mujib Dhaka Marathon?"
answer_question(question)

Processing subject: who
Matching node found: (_16:Subject {name: 'who'})
No outgoing relationships for node (_16:Subject {name: 'who'})


In [ ]:
def print_graph_info():
    # Connect to your Neo4j database
    graph = Graph("neo4j+s://a6801d44.databases.neo4j.io", user="neo4j", password="049h8PxTvW314YzhT1mxjNfkLpTLewtTG7FEZMj0vwY")

    # Create a NodeMatcher
    matcher = NodeMatcher(graph)

    # Print information about all relationships in the graph
    for rel in graph.relationships:
        try:
            # Check if the relationship has a type attribute (to filter out integers)
            if hasattr(rel, 'type'):
                # Print details of the relationship, including type and properties
                print(f"Relationship: {rel.type}, Properties: {dict(rel)}")

                # Attempt to retrieve start and end nodes
                start_node = matcher.match(labels=rel.start_node.labels, name=rel.start_node['name']).first()
                end_node = matcher.match(labels=rel.end_node.labels, name=rel.end_node['name']).first()

                # Check if both start and end nodes are valid
                if start_node and end_node:
                    print(f"  Start node: {start_node}")
                    print(f"  End node: {end_node}")
                else:
                    print(f"  Invalid relationship detected: {rel}")
        except Exception as e:
            print(f"Error processing relationship: {e}")

# Print information about the graph
print_graph_info()